# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.11.4.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [1]:
!poetry install

Installing dependencies from lock file

No dependencies to install or update


In [2]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [3]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.


In [4]:
text_loader = TextFileLoader("data/PMarcaBlogs.txt")
documents = text_loader.load_documents()
len(documents)

1

In [5]:
print(documents[0][:100])


The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horow


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [6]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

373

Let's take a look at some of the documents we've managed to split.

In [7]:
split_documents[0:1]

['\ufeff\nThe Pmarca Blog Archives\n(select posts from 2007-2009)\nMarc Andreessen\ncopyright: Andreessen Horowitz\ncover design: Jessica Hagy\nproduced using: Pressbooks\nContents\nTHE PMARCA GUIDE TO STARTUPS\nPart 1: Why not to do a startup 2\nPart 2: When the VCs say "no" 10\nPart 3: "But I don\'t know any VCs!" 18\nPart 4: The only thing that matters 25\nPart 5: The Moby Dick theory of big companies 33\nPart 6: How much funding is too little? Too much? 41\nPart 7: Why a startup\'s initial business plan doesn\'t\nmatter that much\n49\nTHE PMARCA GUIDE TO HIRING\nPart 8: Hiring, managing, promoting, and Dring\nexecutives\n54\nPart 9: How to hire a professional CEO 68\nHow to hire the best people you\'ve ever worked\nwith\n69\nTHE PMARCA GUIDE TO BIG COMPANIES\nPart 1: Turnaround! 82\nPart 2: Retaining great people 86\nTHE PMARCA GUIDE TO CAREER, PRODUCTIVITY,\nAND SOME OTHER THINGS\nIntroduction 97\nPart 1: Opportunity 99\nPart 2: Skills and education 107\nPart 3: Where to go and wh

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [8]:
import os
import openai
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    openai.api_key = getpass("OpenAI API Key: ")
    os.environ["OPENAI_API_KEY"] = openai.api_key
else:
    openai.api_key = os.environ["OPENAI_API_KEY"]

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #2!

#### 🔍Answer #1:
> 1. Is there any way to modify this dimension?

To modify the dimension of `text-embedding-3-small`, you can pass the `dimensions` parameter with the desired value when using the OpenAI API. For example, you can set the dimension to 1024 by passing `dimensions=1024` in the API request.


> 2. What technique does OpenAI use to achieve this?

OpenAI uses a technique called
[`decimation`](https://community.openai.com/t/embedding-models-dimension/308856)
to shorten the embeddings without losing their concept-representing properties.
Decimation involves removing some numbers from the end of the sequence, which
allows developers to trade-off performance and cost of using embeddings.

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [9]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

#### 🔍Answer #2:

`async` is more efficient because it allows for concurrent execution of tasks,
which can save time and improve performance. Essentially, all IO bound tasks
should be done in `async` mode.

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [10]:
vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)

[('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\nproduct management executive. The CEO who used to be in\nsales who has a weak sales executive. The CEO who used to be\nin marketing who has a weak marketing executive.\nI call this the “Michael Eisner Memorial Weak Executive Problem” — aaer the CEO of Disney who had previously been a brilliant TV network executive. When he bought ABC at Disney, it\npromptly fell to fourth place. His response? “If I had an extra\ntwo days a week, I could turn around ABC myself.” Well, guess\nwhat, he didn’t have an extra two days a week.\nA CEO — or a startup founder — oaen has a hard time letting\ngo of the function that brought him to the party. The result: you\nhire someone weak into the executive role for that function so\nthat you can continue to b

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4o-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

#### 🔍Answer #3:



One of the main methods for generating more consistent outputs is using the `seed` parameter[1][2]:

- Set the `seed` parameter to an integer value of your choice in your API request
- Use the same `seed` value across requests where you want consistent outputs
- Keep all other parameters (prompt, temperature, top_p, etc.) the same across requests


The API response includes a `system_fingerprint` field[2]. This identifies the current model configuration used to generate the completion. To get consistent outputs:

- Check that the `system_fingerprint` matches across requests
- If it differs, you may see different outputs due to backend changes

Additional Considerations:

- Use lower temperature and top_p values to reduce randomness[1]
- Request structured outputs (e.g. JSON) to constrain the response format[3]
- Consider fine-tuning a model on your specific use case for more consistent outputs[3]
- Be aware that even with these measures, there's still a small chance of variation in outputs due to the inherent nature of language models[2]

By implementing these techniques, you can significantly improve the reproducibility of outputs from the OpenAI API, though perfect determinism is not guaranteed.

Citations:

- [1] https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/reproducible-output
- [2] https://cookbook.openai.com/examples/reproducible_outputs_with_the_seed_parameter
- [3] https://community.openai.com/t/coping-with-inconsistent-results-on-identical-inputs/576342

### Creating and Prompting OpenAI's `gpt-4o-mini`!

Let's tie all these together and use it to prompt `gpt-4o-mini`!

In [11]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [12]:
print(response)

The best way to write a loop in Python largely depends on the specific task you want to accomplish. However, I'll outline a few common types of loops and best practices to help you choose the right approach.

### 1. Using a `for` loop

A `for` loop is commonly used when you know the number of iterations in advance or when you are iterating over a sequence (like a list, tuple, string, or dictionary).

```python
# Example: Iterate through a list
my_list = [1, 2, 3, 4, 5]
for item in my_list:
    print(item)
```

### 2. Using a `while` loop

A `while` loop is useful when the number of iterations is not known ahead of time. The loop continues until a specified condition is no longer true.

```python
# Example: While loop example
count = 0
while count < 5:
    print(count)
    count += 1
```

### 3. Using list comprehensions

For creating new lists based on existing lists, list comprehensions are often the cleanest and most Pythonic way.

```python
# Example: Squaring numbers in a list
squa

## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [13]:
RAG_PROMPT_TEMPLATE = """ \
Use the provided context to answer the user's query.

You may not answer the user's query unless there is specific context in the following text.

If you do not know the answer, or cannot answer, please respond with "I don't know".
"""

rag_prompt = SystemRolePrompt(RAG_PROMPT_TEMPLATE)

USER_PROMPT_TEMPLATE = """ \
Context:
{context}

User Query:
{user_query}
"""


user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)

        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = rag_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)

        return {"response" : self.llm.run([formatted_system_prompt, formatted_user_prompt]), "context" : context_list}

#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through the Week 1 Day 1 "Prompting OpenAI Like A Developer" material for an answer to this question!

#### 🔍Answer #4:
To encourage more thoughtful and detailed responses from an LLM in a Retrieval Augmented Generation (RAG) application, you can employ several prompting strategies:

1. Explicit instructions for depth: Include clear directives in your system prompt asking for thorough, detailed responses[1]. For example:

> Provide a comprehensive and detailed answer, incorporating relevant information
> from the given context. Explain concepts thoroughly and give examples where
> appropriate.

2. Encourage analytical thinking: Ask the LLM to analyze the retrieved
  information critically[1]. For instance:
  
  > Carefully analyze the provided context. Compare and contrast different
  > viewpoints if present, and explain the reasoning behind your conclusions.

  You can use some of the well-known human reasoning strategies to guide the LLM
  to think step-by-step. These strategies are included at the end of this notebook file.

3. Request multi-step reasoning: Prompt the LLM to break down its thought process[2]:

  > Approach this question step-by-step. First, summarize the key points from
  > the context. Then, apply that information to address the user's query.
  > Finally, draw a well-reasoned conclusion.

4. Specify output structure: Guide the LLM to organize its response in a structured manner[5]:

  > Structure your response as follows:
  >
  >  1. Brief overview
  >  2. Key points from the context
  >  3. Detailed explanation
  >  4. Examples or applications
  >  5. Conclusion

  It might be even better to give the LLM output structure as a `spec` file ; i.e
  use JSON schema to define the output structure.[9]

5. Encourage source citation: Ask the LLM to reference the retrieved information[1]:

  > When using information from the provided context, indicate the source by
  > mentioning 'According to [source]' or similar phrasing.

6. Role-playing: Assign a specific expert role to the LLM to encourage more authoritative and detailed responses[2]:

  > Assume the role of a leading expert in [relevant field]. Provide an in-depth
  > explanation as you would to a colleague.

7. Follow-up questions: Include prompts for the LLM to ask itself follow-up questions[2]:

  > After providing your initial response, ask yourself two relevant follow-up
  > questions and answer them to provide additional depth and clarity.

8. Contextual relevance: Explicitly instruct the LLM to focus on the most relevant parts of the retrieved context[1]:

  > Carefully consider the retrieved information and focus on the aspects most
  > directly relevant to the user's query. Explain why you've chosen to
  > highlight certain information.


Citations:

- [1] https://www.snowflake.com/guides/retrieval-augmented-generation-improving-llm-outputs/
- [2] https://www.k2view.com/blog/rag-prompt-engineering/
- [3] https://www.mdpi.com/2504-4990/6/1/18
- [4] https://arxiv.org/html/2404.06910v1
- [5] https://datarootlabs.com/blog/prompting-techniques
- [6] https://www.superannotate.com/blog/llm-prompting-tricks
- [7] https://www.qwak.com/post/prompt-management
- [8] https://stackoverflow.blog/2023/10/18/retrieval-augmented-generation-keeping-llms-relevant-and-current/]
- [9] https://json-schema.org/understanding-json-schema

In [14]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)

In [15]:
retrieval_augmented_qa_pipeline.run_pipeline("What is the 'Michael Eisner Memorial Weak Executive Problem'?")

{'response': "The 'Michael Eisner Memorial Weak Executive Problem' refers to the tendency of CEOs or startup founders to hire executives who are weak in the functions in which the CEO has expertise. This often happens because the CEO has difficulty letting go of the area that initially contributed to their success. As an example, the text mentions Michael Eisner, who was a successful TV network executive but faced challenges when he hired a weak executive to run ABC after acquiring it. The concept highlights the risk in hiring someone inferior in judgment or skill so that the CEO can maintain control and influence over that function.",
 'context': [('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\nproduct management executive. The CEO who used to be in\nsales who has a weak sales execu

### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database

While these are suggestions, you should feel free to make whatever augmentations you desire! 

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

In [16]:
import PyPDF2
import os


class TextFileLoader:
    def __init__(self, path: str, encoding: str = "utf-8"):
        self.documents = []
        self.path = path
        self.encoding = encoding

    def load(self):
        if os.path.isdir(self.path):
            self.load_directory()
        # TODO: it would be better to just check and see if the file is a binary
        # blob or not; in case it is a binary blob ; then check for PDF
        # extension; otherwise , load it as a text file.
        elif os.path.isfile(self.path):
            if self.path.endswith(".txt"):
                self.load_file()
            elif self.path.endswith(".pdf"):
                self.load_pdf()
            else:
                raise ValueError(
                    "Unsupported file type. Only .txt and .pdf files are supported."
                )
        else:
            raise ValueError(
                "Provided path is neither a valid directory nor a supported file."
            )

    def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())

    def load_pdf(self):
        with open(self.path, "rb") as f:
            pdf_reader = PyPDF2.PdfReader(f)
            text = "".join(page.extract_text() for page in pdf_reader.pages)
            self.documents.append(text)

    def load_directory(self):
        for root, _, files in os.walk(self.path):
            for file in files:
                if file.endswith((".txt", ".pdf")):
                    file_path = os.path.join(root, file)
                    if file.endswith(".txt"):
                        with open(file_path, "r", encoding=self.encoding) as f:
                            self.documents.append(f.read())
                    elif file.endswith(".pdf"):
                        with open(file_path, "rb") as f:
                            pdf_reader = PyPDF2.PdfReader(f)
                            text = "".join(
                                page.extract_text() for page in pdf_reader.pages
                            )
                            self.documents.append(text)

    def load_documents(self):
        self.load()
        return self.documents

In [17]:
pdf_text_loader = TextFileLoader("data/well-defined-human-reasoning-approaches.pdf")
pdf_documents = pdf_text_loader.load_documents()
len(pdf_documents)

1

In [18]:
print(pdf_documents[0][:100])

W ell-Defined Human Reasoning Approaches
1. Deductive Reasoning
Origin: Ancient Greece, formalized b


In [19]:
pdf_text_splitter = CharacterTextSplitter()
pdf_split_documents = pdf_text_splitter.split_texts(pdf_documents)
len(pdf_split_documents)

35

In [20]:
pdf_split_documents[0:1]


['W ell-Defined Human Reasoning Approaches\n1. Deductive Reasoning\nOrigin: Ancient Greece, formalized by Aristotle (384-322 BCE)\nExplanation: Deductive reasoning starts with a general statement or hypoth-\nesis and examines the possibilities to reach a specific, logical conclusion. The\nconclusion is derived from the given premises and must be true if the premises\nare true.\nSynonyms: Logical reasoning, top-down reasoning\nT asks for LLMs:\n• Solving logic puzzles\n• Debugging code\n• Analyzing arguments for validity\n• Making predictions based on given rules\nExample prompt: “Using deductive reasoning, determine if the conclusion\n‘Socrates is mortal’ is valid based on the premises ‘All men are mortal’ and\n‘Socrates is a man’ . ”\n2. Inductive Reasoning\nOrigin: Ancient Greece, developed by philosophers like Aristotle\nExplanation: Inductive reasoning makes broad generalizations from specific\nobservations. It involves moving from specific instances to a general conclusion,\nwhich

In [21]:
pdf_vector_db = VectorDatabase()
pdf_vector_db = asyncio.run(pdf_vector_db.abuild_from_list(pdf_split_documents))

In [22]:
pdf_vector_db.search_by_text("What are the human reasoning approaches that were invented by the greeks?", k=3)

[('W ell-Defined Human Reasoning Approaches\n1. Deductive Reasoning\nOrigin: Ancient Greece, formalized by Aristotle (384-322 BCE)\nExplanation: Deductive reasoning starts with a general statement or hypoth-\nesis and examines the possibilities to reach a specific, logical conclusion. The\nconclusion is derived from the given premises and must be true if the premises\nare true.\nSynonyms: Logical reasoning, top-down reasoning\nT asks for LLMs:\n• Solving logic puzzles\n• Debugging code\n• Analyzing arguments for validity\n• Making predictions based on given rules\nExample prompt: “Using deductive reasoning, determine if the conclusion\n‘Socrates is mortal’ is valid based on the premises ‘All men are mortal’ and\n‘Socrates is a man’ . ”\n2. Inductive Reasoning\nOrigin: Ancient Greece, developed by philosophers like Aristotle\nExplanation: Inductive reasoning makes broad generalizations from specific\nobservations. It involves moving from specific instances to a general conclusion,\nwhic

In [23]:
pdf_retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=pdf_vector_db,
    llm=chat_openai
)

In [24]:
response=pdf_retrieval_augmented_qa_pipeline.run_pipeline("give me 2 examples for each human reasoning approaches that were invinted by the greeks")

In [ ]:
print(f'{response["response"]}')

1. **Deductive Reasoning:**
   - Example 1: All humans are mortal. Socrates is a human. Therefore, Socrates is mortal.
   - Example 2: All birds have feathers. A sparrow is a bird. Therefore, a sparrow has feathers.

2. **Inductive Reasoning:**
   - Example 1: The sun has risen in the east every day observed. Therefore, the sun will rise in the east tomorrow.
   - Example 2: Most students I’ve met at this university study hard. Therefore, all students at this university study hard.

3. **Dialectical Reasoning:**
   - Example 1: Thesis: School uniforms improve discipline. Antithesis: School uniforms limit personal expression. Synthesis: While school uniforms may increase discipline, incorporating some aspects of personal choice can balance expression and order.
   - Example 2: Thesis: Technology isolates people. Antithesis: Technology connects people. Synthesis: Technology can both isolate and connect, depending on how it is used in social interactions.
